# Amazon product catalog


## Problem scope

What do Amazon reviews say about the product, and can reviews be used reliably to predict the product category?

## Open questions / workflow

1. Predict the rating based on item desc.: regression w/ language data

2. How well reviewed something is

3. Figure out product, product contents, product tags, document per row + brand + company type, product category, description

4. Probability that the thing we labeled is actually in that class?


## Data imports

In [1]:
#libraries
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns, re

### Product dataset

In [2]:
#import ecom dataset
ecom = pd.read_csv('../data/amazon_co-ecommerce_sample.csv')

In [3]:
#check data size
ecom.shape

(10000, 17)

In [4]:
#check data types
ecom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   uniq_id                                      10000 non-null  object 
 1   product_name                                 10000 non-null  object 
 2   manufacturer                                 9993 non-null   object 
 3   price                                        8565 non-null   object 
 4   number_available_in_stock                    7500 non-null   object 
 5   number_of_reviews                            9982 non-null   object 
 6   number_of_answered_questions                 9235 non-null   float64
 7   average_review_rating                        9982 non-null   object 
 8   amazon_category_and_sub_category             9310 non-null   object 
 9   customers_who_bought_this_item_also_bought   8938 non-null   object 
 10 

In [5]:
#peek at dataframe
ecom.head()

,uniq_id,product_name,manufacturer,price,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,sellers
0,eac7efa5dbd3d667f26eb3d3ab504464,Hornby 2014 Catalogue,Hornby,£3.42,5 new,15,1.0,4.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Product Description Hornby 2014 Catalogue Box ...,Technical Details Item Weight640 g Product Dim...,Product Description Hornby 2014 Catalogue Box ...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Does this catalogue detail all the previous Ho...,Worth Buying For The Pictures Alone (As Ever) ...,"{""seller""=>[{""Seller_name_1""=>""Amazon.co.uk"", ..."
1,b17540ef7e86e461d37f3ae58b7b72ac,FunkyBuys® Large Christmas Holiday Express Fes...,FunkyBuys,£16.99,NaN,2,1.0,4.5 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,Size Name:Large FunkyBuys® Large Christmas Hol...,Technical Details Manufacturer recommended age...,Size Name:Large FunkyBuys® Large Christmas Hol...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,can you turn off sounds // hi no you cant turn...,Four Stars // 4.0 // 18 Dec. 2015 // By\n \...,"{""seller""=>{""Seller_name_1""=>""UHD WHOLESALE"", ..."
2,348f344247b0c1a935b1223072ef9d8a,CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...,ccf,£9.99,2 new,17,2.0,3.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Classic-Train-Lights-B...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,Technical Details Manufacturer recommended age...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,http://www.amazon.co.uk/Train-With-Tracks-Batt...,What is the gauge of the track // Hi Paul.Trut...,**Highly Recommended!** // 5.0 // 26 May 2015 ...,"{""seller""=>[{""Seller_name_1""=>""DEAL-BOX"", ""Sel..."
3,e12b92dbb8eaee78b22965d2a9bbbd9f,HORNBY Coach R4410A BR Hawksworth Corridor 3rd,Hornby,£39.99,NaN,1,2.0,5.0 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,NaN,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,Technical Details Item Weight259 g Product Dim...,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,NaN,NaN,I love it // 5.0 // 22 July 2013 // By\n \n...,NaN
4,e33a9adeed5f36840ccc227db4682a36,Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...,Hornby,£32.19,NaN,3,2.0,4.7 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R6367-RailRoad-...,Product Description Hornby RailRoad 0-4-0 Gild...,Technical Details Item Weight159 g Product Dim...,Product Description Hornby RailRoad 0-4-0 Gild...,http://www.amazon.co.uk/Hornby-R2672-RailRoad-...,NaN,Birthday present // 5.0 // 14 April 2014 // By...,NaN


## Cleaning

Let's clean up our columns, including price, so that they can converge properly.

### Remove extraneous characters

From the pound signs in the prices, to prices listed as ranges, we have some issues in our data formats. Let's fix these.

For price ranges, I will merely consume the lower bound of the price.

In [6]:
#Substantial datatype conversion help here from https://pbpython.com/currency-cleanup.html

def clean_price(x):
    
    if isinstance(x, str):
#If the value is a string, then remove currency symbol, delimiters and anything 
#else following the price; otherwise, the value is numeric and can be converted as is.
        
        return(x.replace('£', '').replace(',', '').split(' - ')[0])
    #strip price of the pound sign
    #we need to get rid of price ranges. I will just consume the LOWER BOUND in case of a price range.

    return(x)

In [7]:
#apply my price function to convert the price column type
ecom['price'] = ecom['price'].apply(clean_price).astype('float')

In [8]:
#strip out "out of 5 stars" from average_review_rating
ecom['average_review_rating'] = ecom['average_review_rating'].str.strip(' out of 5 stars')
ecom['average_review_rating'].head(2)

0    4.9
1     4.
Name: average_review_rating, dtype: object

In [9]:
#strip out the word "new" from number in stock -- but not actually useful for training, so can skip this?

In [10]:
#same thing but to apply more ubiquitously to other columns
#goal is to filter out non-num values

def clean_num(x):
    
    if not isinstance(x, str): #I basically need something like a "not like 0123456789" and loop through str...
         return x #why isn't it working to check for nums like it did for price above?
    else:
        return 0

In [11]:
#strip out commas from number_of_reviews column
ecom['number_of_reviews'] = ecom['number_of_reviews'].str.strip(",")
ecom['number_of_reviews'] = ecom['number_of_reviews'].replace(',', '').replace("'","")

#df[pd.to_numeric(df['column'], errors='coerce').notnull()]
ecom['number_of_reviews'] = ecom[pd.to_numeric(ecom['number_of_reviews'], errors='coerce').notnull()]

#but really, I want to IGNORE (ie FILTER OUT) those obs. - use `isnumeric`
#from: https://stackoverflow.com/questions/33961028/remove-non-numeric-rows-in-one-column-with-pandas

#df_clean = df[df.id.str.isnumeric()]
#ecom['number_of_reviews'] = ecom[ecom['number_of_reviews'].str.isnumeric()] 

#NOTE: be sure to run `.dropna()` on this column first!

In [14]:
ecom['number_of_reviews'] = ecom['number_of_reviews'].apply(clean_num).astype('int')

#alt
#ecom['number_of_reviews'] = ecom[ecom['number_of_reviews'].apply(lambda x: isinstance(x, (int, np.int64)))]

In [15]:
ecom['number_of_reviews'].sort_values(ascending=False)[:30]

9999    0
3331    0
3338    0
3337    0
3336    0
3335    0
3334    0
3333    0
3332    0
3330    0
3321    0
3329    0
3328    0
3327    0
3326    0
3325    0
3324    0
3323    0
3339    0
3340    0
3341    0
3342    0
3357    0
3356    0
3355    0
3354    0
3353    0
3352    0
3351    0
3350    0
Name: number_of_reviews, dtype: int64

### Converting object columns to numeric where applicable

We had 4 columns originally that seemed like good candidates for conversion to numeric:

* price
* number_of_reviews
* number_of_answered_questions
* average_review_rating

In [ ]:
#force num type on number_of_reviews, number_of_answered_questions, price, average_review_rating
ecom['number_of_answered_questions'] = ecom[['number_of_answered_questions']].apply(pd.to_numeric)

In [ ]:
ecom['average_review_rating'] = ecom['average_review_rating'].astype(float)

In [ ]:
#locate the issue with the number of reviews not converting
ecom.loc[ecom['number_of_reviews']=='1,040']

In [ ]:
#ecom['number_of_reviews'] = ecom[ecom['number_of_reviews'] != '1,040'] #not necessary once we've successfully removed chars

In [ ]:
#ecom['number_of_reviews'] = ecom[['number_of_reviews']].apply(pd.to_numeric)
ecom['number_of_reviews'] = ecom['number_of_reviews'].astype(int)

In [ ]:
#confirm conversions
ecom.dtypes

### Imputing nulls

For most of the fields, imputing with a 0 seems to make sense -- no reviews is no different than 0 reviews.

For price, however, we'll want to impute with the mean, now that we've successfully converted it to numeric.

As for average review rating, in the absence of one, we will simply drop those few observations since that's a valuable column and I'd want to be careful extending any kind of mean to it, since it is possibly our target.

In [13]:
#`dropna.()` nulls without an average_review_rating since we need those for training, and there are few
ecom.dropna(subset=['average_review_rating'], inplace=True)
ecom.dropna(subset=['number_of_reviews'], inplace=True)

In [ ]:
#fill price with average of column
ecom['price'] = ecom['price'].fillna(ecom['price'].mean())

In [ ]:
#fill in NaN's with 0's for everything else
ecom = ecom.fillna(0)
ecom.head(3)

In [ ]:
ecom.isnull().sum()

## EDA

In [ ]:
ecom.columns

In [ ]:
ecom['price'].value_counts().sort_values(ascending=False)[:15]
#do as dist, instead

In [ ]:
ecom.describe()

In [ ]:
ecom['number_available_in_stock'].value_counts()[:10]
#replace w/ a search for ~like new, ~like used, and then get a count

## Feature engineering

We need to convert things like ratings so that the modeling works correctly.

The plan:

* Categorize / one hot encode the ratings column 

In [ ]:
#one-hot encode ratings column on a scale of 1-5? Or is target?

In [ ]:
#move out new vs. used items into sep. columns (binarize)

## NLP

In [ ]:
#vectorize text

## Modeling - classifier

In [ ]:
#reviews
#price
#category

## Evaluation

## Conclusions & next steps